## Exibe logs para modelos treinados

In [9]:
import os 
import pandas as pd
# print("Operations History:        \n", operationsHistory.to_string(index=False))

path = "../Results/test/logs/class"
files = os.listdir(path)
for file in files:
    if file.endswith(".csv"):
        print(f"================================================ {file} ===========================================================")
        df = pd.read_csv(path+"/"+file, sep=";")
        print(df.to_string(index=False), end="\n\n")


================================================ PETR4_B_0_30min_logs.csv ===========================================================
                  model  qtdBuying  inirialValue  finalValue  percentual  accuracy       f1  truePositives  trueNegatives  falsePositives  falseNegatives
                    SVM          1          1000 1310.886644   31.088664  0.537472 0.699163            0.0          710.0           611.0             0.0
                    KNN        223          1000 1411.542006   41.154201  0.501893 0.614302          139.0          524.0           472.0           186.0
                     LR          1          1000 1310.886644   31.088664  0.537472 0.699163            0.0          710.0           611.0             0.0
                   LSTM          5          1000 1171.470821   17.147082  0.503407 0.604819          163.0          502.0           448.0           208.0
                    MLP          8          1000  989.771856   -1.022814  0.498864 0.594859     

## Exibe logs para estratégias implementadas

In [10]:
import os 
import pandas as pd
# print("Operations History:        \n", operationsHistory.to_string(index=False))

path = "../Results/test/logs/economic"
files = os.listdir(path)
for file in files:
    if file.endswith(".csv"):
        print(f"================================================ {file} ===========================================================")
        df = pd.read_csv(path+"/"+file, sep=";")
        print(df.to_string(index=False), end="\n\n")

================================================ ensamble2_buying.csv ===========================================================
                        model  qtdBuying  inirialValue  finalValue  percentual
    strat_TCC_PETR4_B_0_30min         48          1000 1347.181726   34.718173
strat_hurwicz_PETR4_B_0_30min          1          1000 1290.607735   29.060773
                 Buy and Hold          1          1000 1310.886644   31.088664
    strat_TCC_PETR4_B_0_15min        572          1000  952.374055   -4.762595
strat_hurwicz_PETR4_B_0_15min          1          1000  794.938918  -20.506108
                 Buy and Hold          1          1000  801.231310  -19.876869
    strat_TCC_PETR4_B_0_60min        139          1000  591.442413  -40.855759
strat_hurwicz_PETR4_B_0_60min          0          1000 1000.000000    0.000000
                 Buy and Hold          1          1000  513.129103  -48.687090

================================================ regression_buying.csv ========